# Pure State Tomography

### Junling Long, Adrian Tan, Sven Jandura

# Introduction

 Suppose we have some quantum circuit, which generates a state |psi>. We are interested in the coefficients of psi.
 Unfortunatly, due to noise effects, on a real quantum computer we can only generate rho = (1-epsilon)|psi><psi| + epsilon rho_error
 with epsilon << 1. How do we find a good estimate of |psi>

 Using usual quantum state tomography, we can could find rho and would estimate |psi> to be the eigenvector of rho which belongs to
 the larges eigenvalue. Using pure state tomography, we can find an estimate of |psi> directly from the measurements. This requires 
 fewer operators to be measured then for a full state tomography for the same accuraccy.


# Import

In [39]:
import numpy as np

import qiskit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise
import verification.tomography as tomo

from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise

from verification.tomography.fitters.pure_state_mle_fit import pure_state_mle_fit, pure_state_mle_fit_density_matrix

from scipy.linalg import sqrtm


# Testing the protocol with near pure states for two qubits

## 1. Generating test density matrix

Consider the Bell states |psi1> = 1/sqrt(2)(|00>+|11>) and |psi2> = 1/sqrt(2)(|00>-|11>) and rho = (1-epsilon)|psi1><psi1| + epsilon|psi2><psi2|



In [40]:
#Psi1
# Setup circuit to generate |psi1>
qr1 = QuantumRegister(2)
bell1 = QuantumCircuit(qr1)
bell1.h(qr1[0])
bell1.cx(qr1[0], qr1[1])

# get state |psi1> using statevector_simulator
job = qiskit.execute(bell1, Aer.get_backend('statevector_simulator'))
psi1 = job.result().get_statevector(bell1)
print("|psi1> = {}".format(psi1))

# get expectation values of operators ... and convert the to the right format
qst_bell1 = tomo.state_tomography_circuits(bell1, qr1)
job = qiskit.execute(qst_bell1, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi1 = tomo.tomography_data(job.result(), qst_bell1)
probs_psi1, basis_matrix, _ = tomo.fitter_data(tomo_counts_psi1)

#Psi 2
# Setup circuit to generate |psi2>
qr2 = QuantumRegister(2)
bell2 = QuantumCircuit(qr2)
bell2.y(qr2[0])
bell2.h(qr2[0])
bell2.cx(qr2[0], qr2[1])

# get state |psi2> using statevector_simulator
job = qiskit.execute(bell2, Aer.get_backend('statevector_simulator'))
psi2 = job.result().get_statevector(bell2)
print("|psi2> = {}".format(psi2))


# get expectation values of operators ... and convert the to the right format
qst_bell2 = tomo.state_tomography_circuits(bell2, qr2)
job = qiskit.execute(qst_bell2, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi2 = tomo.tomography_data(job.result(), qst_bell2)
probs_psi2, _ , _ = tomo.fitter_data(tomo_counts_psi2) # the basis_matrix is the same for both states


|psi1> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|psi2> = [0.+0.70710678j 0.+0.j         0.+0.j         0.-0.70710678j]


Generate probabilities of rho

In [41]:
epsilon = 0.3
probs_rho = [(1-epsilon)*probs_psi1[i]+epsilon*probs_psi2[i] for i in range(len(probs_psi1))]

function for calculating inner product between states


In [42]:
def inner_product(psi, phi):
    return np.abs(sum([psi[i].conj()*phi[i] for i in range(len(psi))]))**2

## 2. Unconstrained state tomography (maximum likelyhood approach)


In [43]:
rho_full_mle = tomo.state_mle_fit(probs_rho, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi1, eigenvectors[0])))

Eigenvalues of rho: [9.95527140e-01+1.78177672e-19j 4.47286010e-03+1.38448978e-17j
 3.71209356e-18-2.76327455e-17j 2.62171652e-18+1.40808629e-17j]
Guess for psi: [ 0.70782213+0.j         -0.41025506+0.21629985j  0.03305804+0.4414323j
  0.30843415-0.0972833j ]
Fidelity: 0.5211204350281934


## 3. Pure state tomography (maximum likelyhood approach)


In [12]:
psi_guess,_ = pure_state_mle_fit(probs_rho, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi1, psi_guess)))


n:  4
Guess for psi: [(0.7065192289256829-0.000913718574473898j), (0.0005293206996994222-0.007033363698076963j), (-0.002787057982869528-0.005772105551600478j), (0.7076170399240566+0.004127445424225231j)]
Fidelity: 0.9998958574582636


# Testing the protocol with standard noise model with two qubits

## 1. Set up random circuit

In [13]:
qr = QuantumRegister(2)
circuit = QuantumCircuit(qr)
params = np.random.rand(12)
circuit.u3(params[0], params[1], params[2], qr[0])
circuit.u3(params[3], params[4], params[5], qr[1])
circuit.cx(qr[0], qr[1])
circuit.u3(params[6], params[7], params[8], qr[0])
circuit.u3(params[9], params[10], params[11], qr[1])


job = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(circuit)
print("psi: {}".format(psi))

psi: [ 0.92110659-0.03878696j  0.07763912+0.02760747j  0.20670353+0.28109052j
 -0.14659281-0.00636737j]


## 2. Apply standard noise

In [15]:
#IBMQ.load_accounts()
IBMQ.enable_account('a6140115a9d2692b8a711d0f31fdc92b7ae793865719445a78fc210220de52765da18d0eab774d245fbc9e5f40c94e99d9c536f9f1749acd7b902b3bd9931dd5')
device = IBMQ.get_backend('ibmqx4')
properties = device.properties()
coupling_map = device.configuration().coupling_map

gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678)]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)


# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')


/Users/atanteck/anaconda3/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqprovider.py:251: UserWarning: Credentials are already in use.
  warnings.warn('Credentials are already in use.')


## 3. Execute

In [17]:
qst= tomo.state_tomography_circuits(circuit, qr)
job = qiskit.execute(qst, Aer.get_backend('qasm_simulator'),noise_model=noise_model, coupling_map=coupling_map, basis_gates=basis_gates, shots=5000)
tomo_counts = tomo.tomography_data(job.result(), qst)
probs_psi, basis_matrix, _ = tomo.fitter_data(tomo_counts)

## 4. Unconstrained state tomography using IBM (maximum likelyhood approach)


In [18]:
rho_full_mle = tomo.state_mle_fit(probs_psi, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi, eigenvectors[0])))

Eigenvalues of rho: [0.84447043+1.66921615e-18j 0.01255959+5.86184854e-18j
 0.0780254 +1.05446329e-17j 0.06494459+1.00626069e-18j]
Guess for psi: [ 0.91884604+0.j          0.08858412-0.07626315j -0.19775502+0.3140015j
 -0.05961891+0.02829029j]
Fidelity: 0.8432986875465517


## 5. Pure state tomography (maximum likelyhood approach)

In [19]:
psi_guess,_ = pure_state_mle_fit(probs_psi, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi, psi_guess)))

n:  4
Guess for psi: [(0.915327160486264+0.013787938310871389j), (0.09121478523797215+0.026805571036491404j), (0.19559670226572515+0.3036947360393523j), (-0.14955599251314677-0.009583778840100195j)]
Fidelity: 0.9995048485605685


# Testing the protocol with  pure states for three qubits using expectation values

In [31]:
def get_expectation_values(tomo_counts,nbits, shots=5000):
    #operators = ['II', 'IX', 'IY', 'IZ', 'XI', 'YX', 'YY', 'YZ', 'ZX', 'ZY', 'ZZ']
    operators = ['IIX','IIY','IIZ','IXI','IXX','IXY','IYI','IYX','IYY','IZI','XIZ','XXX','XXY','XYX','XYY','XZX','XZY','YXX','YXY','YXZ','YYX','YYY','YYZ','YZI','ZII','ZXZ','ZYZ','ZZX','ZZY','ZZZ','III']
    #operators = ['II', 'IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YX', 'YY', 'YZ', 'ZX', 'ZY', 'ZZ']
    #operators = ['XY']
    probs = []
    basis_matrix=[]
    for s in operators:
        #print("Tomo counts: ",tomo_counts)
        prob, mat = tomo.compute_expectation(nbits, s, tomo_counts, shots)
        #print("operator: {}".format(s))
        #print("exp: {}".format(prob))
        #print("mat: {}".format(mat))
        probs.append(prob)
        basis_matrix.append(mat)
    return probs, basis_matrix

In [32]:
def inner_product(psi, phi):
    return np.abs(sum([psi[i].conj()*phi[i] for i in range(len(psi))]))**2

In [33]:
# Setup circuit to generate |psi1>
nbits = 3
qr1 = QuantumRegister(nbits)
bell1 = QuantumCircuit(qr1)

bell1.h(qr1[0])
bell1.cx(qr1[0], qr1[1])
bell1.cx(qr1[1], qr1[2])

# get state |psi1> using statevector_simulator
job = qiskit.execute(bell1, Aer.get_backend('statevector_simulator'))
psi1 = job.result().get_statevector(bell1)

# get expectation values of operators ... and convert the to the right format
qst_bell1 = tomo.state_tomography_circuits(bell1, qr1)
job = qiskit.execute(qst_bell1, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi1 = tomo.tomography_data(job.result(), qst_bell1)
#probs_psi1, basis_matrix, _ = tomo.fitter_data(tomo_counts_psi1)
probs_psi1, basis_matrix = get_expectation_values(tomo_counts_psi1,nbits)


#Psi 2
# Setup circuit to generate |psi2>
qr2 = QuantumRegister(nbits)
bell2 = QuantumCircuit(qr2)
bell2.h(qr2[0])
bell2.cx(qr2[0], qr2[1])
bell2.cx(qr2[1], qr2[2])
# get state |psi2> using statevector_simulator
job = qiskit.execute(bell2, Aer.get_backend('statevector_simulator'))
psi2 = job.result().get_statevector(bell2)
#print("|psi2> = {}".format(psi2))


# get expectation values of operators ... and convert the to the right format
qst_bell2 = tomo.state_tomography_circuits(bell2, qr2)
job = qiskit.execute(qst_bell2, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi2 = tomo.tomography_data(job.result(), qst_bell2)
#probs_psi2, _ , _ = tomo.fitter_data(tomo_counts_psi2) # the basis_matrix is the same for both states
probs_psi2, basis_matrix = get_expectation_values(tomo_counts_psi2,nbits)

# Generate probabilities of rho
epsilon = 0.1
probs_rho = [(1-epsilon)*probs_psi1[i]+epsilon*probs_psi2[i] for i in range(len(probs_psi1))]

In [34]:
# Full state tomography (maximum likelyhood approach)
print("=== IBM state tomography ===")
rho_full_mle = tomo.state_mle_fit(probs_rho, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
#print("Eigenvalues of rho: {}".format(eigenvalues))
#print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi1, eigenvectors[0])))


# Pure state tomography (maximum likelyhood approach)
print("=== Our own pure state tomography ===")
psi_guess,_ = pure_state_mle_fit(probs_rho, basis_matrix)
#print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi1, psi_guess)))

=== IBM state tomography ===
Fidelity: 0.2496073618282664
=== Our own pure state tomography ===
n:  8
Fidelity: 0.999538614920965


In [36]:
print("=== On Noisy simulator ===")
qr = QuantumRegister(nbits)
circuit = QuantumCircuit(qr)

circuit.h(qr[0])
circuit.cx(qr[0], qr[1])
circuit.cx(qr[1], qr[2])

job = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(circuit)

IBMQ.enable_account('a6140115a9d2692b8a711d0f31fdc92b7ae793865719445a78fc210220de52765da18d0eab774d245fbc9e5f40c94e99d9c536f9f1749acd7b902b3bd9931dd5')
device = IBMQ.get_backend('ibmqx4')
properties = device.properties()
coupling_map = device.configuration().coupling_map

gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678)]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)


# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')

#print(noise_model)

qst= tomo.state_tomography_circuits(circuit, qr)
job = qiskit.execute(qst, Aer.get_backend('qasm_simulator'),noise_model=noise_model, coupling_map=coupling_map, basis_gates=basis_gates, shots=5000)
tomo_counts = tomo.tomography_data(job.result(), qst)
#probs_psi, basis_matrix, _ = tomo.fitter_data(tomo_counts)
probs_psi, basis_matrix = get_expectation_values(tomo_counts,nbits)


# Full state tomography (maximum likelyhood approach)
print("=== IBM tomography ===")
rho_full_mle = tomo.state_mle_fit(probs_psi, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
#print("Eigenvalues of rho: {}".format(eigenvalues))
#print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi, eigenvectors[0])))


# Pure state tomography (maximum likelyhood approach)
print("=== Our own pure state tomography ===")
psi_guess,_ = pure_state_mle_fit(probs_psi, basis_matrix)
#print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi, psi_guess)))

=== On Noisy simulator ===
=== IBM tomography ===
Fidelity: 0.22793576331197304
=== Our own pure state tomography ===
n:  8
Fidelity: 0.9980454377317436
